In [1]:
import pickle

import numpy as np
from keras.applications.inception_v3 import InceptionV3,preprocess_input
from keras.layers import Dense,BatchNormalization,Dropout,Embedding,RepeatVector
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential
from keras.models import Model

from pycocotools.coco import COCO
from tqdm import tqdm

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


Using TensorFlow backend.
/mnt/c/Users/Kanishk/Documents/Projects/sem6/wsl/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/mnt/c/Users/Kanishk/Documents/Projects/sem6/wsl/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/mnt/c/Users/Kanishk/Documents/Projects/sem6/wsl/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qi

In [2]:
dataDir='coco'
dataType='train2014'
annFile='{}/annotations/captions_{}.json'.format(dataDir,dataType)

In [3]:
coco=COCO(annFile)

loading annotations into memory...
Done (t=1.46s)
creating index...
index created!


In [4]:
# Since we are using this as feature extractor, the last softmax layer is not useful for us.
inception = InceptionV3(weights='imagenet')

In [ ]:
inception.summary()

In [5]:
# pop the last softmax layer and freezing the remaining layers
inception.layers.pop()

for layer in inception.layers:
    layer.trainable = False

In [6]:
inception.layers[-2].output

<tf.Tensor 'mixed10/concat:0' shape=(None, 8, 8, 2048) dtype=float32>

##### image ==> [Inception + Embedding ] ==> Feature Extracted image of shape (None,300)

In [7]:
# building the final model
final_model = Model(input = inception.input,output = inception.layers[-1].output)

/mnt/c/Users/Kanishk/Documents/Projects/sem6/wsl/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("av...)`
  


In [ ]:
final_model.summary()

In [8]:
TARGET_SIZE = (299,299)

In [9]:
def getImgPath(imgId):
    padding = "0" * (12  - len(str(imgId)))
    imgName = "{}{}".format(padding, imgId)
    return "{}/{}/COCO_{}_{}.jpg".format(dataDir, dataType, dataType, imgName)

In [10]:
getImgPath(1036)


'coco/train2014/COCO_train2014_000000001036.jpg'

In [14]:
# code for image imbedding i.e converting image to 300 dimentional
train_image_extracted = dict()
imgIds = coco.getImgIds()

# imgIds = [151, 260, 307, 404, 450, 491, 514, 529, 575, 671] # dummy list because I don't have all images extracted

try:
    for imgId in tqdm(imgIds, ascii=True, desc="Generating Input Matrix"):
        img = load_img(getImgPath(imgId), target_size=TARGET_SIZE)
        # Converting image to array
        img_array = img_to_array(img)
        nimage = preprocess_input(img_array)
        # Adding one more dimesion
        nimage = np.expand_dims(nimage, axis=0)    
        fea_vec = final_model.predict(nimage)
        train_image_extracted[imgId] = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )


except Exception as e:
    print("Exception got :- \n",e)

Generating Input Matrix:   0%|          | 390/82783 [01:12<4:41:03,  4.89it/s]

Exception got :- 
 image file is truncated (7 bytes not processed)


#### Debugging Images not getting resized
Certain images not extracted properly may give "**image file is truncated (x bytes not processed)**"

Use the below cells to figure out which image it is and re-extract it from the archive.
The `img_id` should be the number visible in progress bar.

##### Example:
PROGRESS BAR:
Generating Input Matrix:   0%|          | **390**/82783 [01:12<4:41:03,  4.89it/s]

set img_id to `390`

In [24]:
img_id = 390
print(imgIds[img_id])
img_path = getImgPath(imgIds[img_id])
print(img_path)
a = load_img(img_path, target_size=TARGET_SIZE)

167126
coco/train2014/COCO_train2014_000000167126.jpg


OSError: image file is truncated (7 bytes not processed)

In [37]:
def save_embedding_matrix(matrix):
    with open("train_image_extracted.pkl","wb") as f:
        pickle.dump(matrix, f)

save_embedding_matrix(train_image_extracted)

In [38]:
print(final_model.output.shape)

(None, 2048)
